# Attitude Dynamics and Control of a Nano-Satellite Orbiting Mars

**Mission Brief**

A nano-satellite in circular Low Mars Orbit (LMO) must autonomously execute attitude control over a 6500 s mission. A second spacecraft (the mothercraft) operates in circular Geosynchronous Mars Orbit (GMO). The nano-satellite must switch between three operational modes:

• Science Mode — point a surface sensor toward Mars (nadir pointing).  
• Power Mode — orient solar panels toward the Sun.  
• Communication Mode — point the antenna toward the mothercraft.

Both spacecraft are modeled using two-body circular orbit dynamics.

The inertial frame is defined as  
$\mathcal{N} = \{\hat{n}_1, \hat{n}_2, \hat{n}_3\}$.

The LMO orbit is described using the passive 3-1-3 Euler sequence  
$(\Omega, i, \theta)$, where  
$\Omega$ = right ascension of ascending node,  
$i$ = inclination,  
$\theta$ = true latitude.

The Hill frame associated with each circular orbit is  

$\mathcal{H} = \{\hat{r}, \hat{\theta}, \hat{h}\}$,

where  
$\hat{r}$ is radial,  
$\hat{\theta}$ is along-track,  
$\hat{h}$ is orbit normal.

For circular two-body motion, the orbital angular rate is constant and given by

$$
\dot{\theta} = \sqrt{\frac{\mu}{r^3}}
$$

where $\mu$ is the Mars gravitational parameter.

Mars has a rotation period of 1 day and 37 minutes. The GMO orbit therefore has the same period and zero inclination (equatorial orbit). Both LMO and GMO angular rates are constant.

The spacecraft body frame is defined as  

$\mathcal{B} = \{\hat{b}_1, \hat{b}_2, \hat{b}_3\}$,

with hardware alignment:
• $+\hat{b}_1$ — science sensor  
• $-\hat{b}_1$ — communication antenna  
• $+\hat{b}_3$ — solar panel normal  

Attitude control is implemented using a PD tracking law:

$$
\mathbf{u}_B = -K\,\boldsymbol{\sigma}_{B/R} - P\,\boldsymbol{\omega}_{B/R}
$$

where $\boldsymbol{\sigma}_{B/R}$ are Modified Rodrigues Parameters and  
$\boldsymbol{\omega}_{B/R}$ is the relative angular velocity.

Spacecraft rotational dynamics follow Euler’s rotational equation of motion:

$$
[I]\dot{\boldsymbol{\omega}} =
-\tilde{\boldsymbol{\omega}}[I]\boldsymbol{\omega} + \mathbf{u}
$$

The objective of this mission simulation is to:

• Generate time-varying orbital reference frames for LMO and GMO.  
• Construct reference attitudes for each operational mode.  
• Compute attitude tracking errors.  
• Integrate full nonlinear attitude dynamics using RK4.  
• Autonomously switch control modes based on eclipse and line-of-sight geometry.  
• Export validated numerical outputs in reproducible text format.

This notebook develops the required kinematics, reference generation, control law implementation, and nonlinear simulation required to complete the mission.
simulation.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

from pathlib import Path

import sys
sys.path.insert(0, r"../../")
import AttitudeKinematicsLib as ak

In [2]:
print("Contents of AttitudeKinematicsLib:")
for name in sorted(dir(ak)):
    if not name.startswith("_"):
        print(name)

Contents of AttitudeKinematicsLib:
BInvmat_CRP
BInvmat_EP
BInvmat_Euler
BInvmat_MRP
BInvmat_PRV
Bmat_CRP
Bmat_EP
Bmat_Euler
Bmat_MRP
Bmat_PRV
CRP
CRP_to_DCM
DCM_to_CRP
DCM_to_EP
DCM_to_Euler
DCM_to_MRP
DCM_to_PRV
DCM_utils
EP_to_DCM
EulerAngles
EulerRodriguesParameters
Euler_to_DCM
MRP
MRP_to_DCM
PRV
PRV_to_DCM
integrate_quaternion
normalize_quat
np
quat_derivative
quat_diff
quat_inv
quat_mult
rotation_matrix_x
rotation_matrix_y
rotation_matrix_z
skew_symmetric
solve_ivp
validate_DCM
validate_vec3
validate_vec4


# Module 1 - Practice Assignment

In [3]:
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

In [4]:
def export_txt(filename_stem: str, data, precision: int = 4) -> None:
    """
    Export mission result data to the outputs directory in Coursera format.

    Parameters
    ----------
    filename_stem : str
        Name of the output file without extension.
    data : scalar | array-like | str
        Numerical data to export. Scalars are written as a single value.
        1D vectors are written as "a b c".
        Strings are written as-is.
    precision : int, optional
        Decimal precision for numerical formatting (default: 12).
    """
    file_path = OUTPUT_DIR / f"{filename_stem}.txt"

    # Case 1: data is a string
    if isinstance(data, str):
        text = data.strip()
        if text.startswith("[") and text.endswith("]"):
            text = text[1:-1].strip()
        text = text.replace(",", " ")
        text = " ".join(text.split())

    # Case 2: numerical data
    else:
        arr = np.asarray(data)

        # Flatten ANY numeric input to 1D row-major
        flat = arr.astype(float).ravel(order="C")

        text = " ".join(f"{val:.{precision}f}" for val in flat)

    file_path.write_text(text, encoding="utf-8")
    print(f"[Saved] {file_path} -> {text}")


In [5]:
part1 = " ".join(str(x) for x in range(1, 10, 2))
print(part1)

export_txt(filename_stem="module1_part1", data=part1)

1 3 5 7 9
[Saved] outputs/module1_part1.txt -> 1 3 5 7 9


In [6]:
A = [[1, 2], 
     [9, 10]]

part2 = " ".join(str(A[i][j]) for i in range(len(A)) for j in range(len(A[0])))
print(part2)

export_txt(filename_stem="module1_part2", data=part2)

1 2 9 10
[Saved] outputs/module1_part2.txt -> 1 2 9 10


In [7]:
part3 = f"{np.pi:.5f}"
print(part3)

export_txt(filename_stem="module1_part3", data=part3)

3.14159
[Saved] outputs/module1_part3.txt -> 3.14159


# Module 2 - Orbits

In [ ]:

R_MARS_km = 3396.19
mu_MARS  = 42828.3

h_km  = 400.0

# LMO
r_LMO_km  = R_MARS_km + h_km
theta_dot_LMO = np.sqrt(mu_MARS/r_LMO_km**3)     # rad/s
Omega_LMO_deg, inc_LMO_deg, theta0_LMO_deg = 20.0, 30.0, 60.0
LMO_angles = np.array([Omega_LMO_deg, inc_LMO_deg, theta0_LMO_deg])

# GMO
r_GMO_km  = 20424.2
theta_dot_GMO = np.sqrt(mu_MARS/r_GMO_km**3)     # rad/s
Omega_GMO_deg, inc_GMO_deg, theta0_GMO_deg = 0.0,  0.0,  250.0
GMO_angles = np.array([Omega_GMO_deg, inc_GMO_deg, theta0_GMO_deg])

# Initial attitude and Body rate
sigma_BN = np.array([0.3, -0.4, 0.5]).T
omega_BN = np.array([np.deg2rad(i) for i in [1.00, 1.75, -2.20]]).T

# Satellite inertia tensor
I = np.array([[10, 0,   0],
              [ 0, 5,   0],
              [ 0, 0, 7.5]])


## Task 1 - Hill-to-Inertial State Transformation for Circular Orbits

<p align="center">
  <img src="images/figure_1.png" width="1000"/>
</p>

The spacecraft is assumed to move in a circular orbit about Mars. Two reference frames are defined:

- The inertial frame $\mathcal{N} = \{\hat{n}_1,\hat{n}_2,\hat{n}_3\}$.
- The Hill (orbit) frame $\mathcal{H} = \{\hat{i}_r,\hat{i}_\theta,\hat{i}_h\}$, where  
  $\hat{i}_r$ is radial,  
  $\hat{i}_\theta$ is along-track, and  
  $\hat{i}_h$ is orbit normal.

For a circular orbit, the position vector is aligned with the radial direction:

$$
\mathbf{r} = r\,\hat{i}_r
\qquad \Rightarrow \qquad
{}^H\mathbf{r} =
\begin{bmatrix}
r \\
0 \\
0
\end{bmatrix}.
$$

To obtain the inertial velocity, we apply the transport theorem:

$$
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{N}}
=
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{H}}
+
\boldsymbol{\omega}_{\mathcal{H}/\mathcal{N}} \times \mathbf{r}.
$$

Since $\mathbf{r}=r\hat{i}_r$,

$$
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{H}}
=
\dot{r}\,\hat{i}_r
+
r\left(\frac{d\hat{i}_r}{dt}\right)_{\mathcal{H}}.
$$

Because $\hat{i}_r$ is fixed in the Hill frame,

$$
\left(\frac{d\hat{i}_r}{dt}\right)_{\mathcal{H}} = \mathbf{0},
$$

so

$$
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{H}} = \dot{r}\,\hat{i}_r.
$$

For a circular orbit, $\dot{r}=0$, and the Hill frame rotates about $\hat{i}_h$ with constant rate

$$
\boldsymbol{\omega}_{\mathcal{H}/\mathcal{N}} = \dot{\theta}\,\hat{i}_h.
$$

Substituting into the transport theorem gives

$$
\dot{\mathbf{r}}
=
r\dot{\theta}\,
(\hat{i}_h \times \hat{i}_r).
$$

Using the Hill-frame right-hand rule $\hat{i}_h \times \hat{i}_r = \hat{i}_\theta$,

$$
\dot{\mathbf{r}} = r\dot{\theta}\,\hat{i}_\theta
\qquad \Rightarrow \qquad
{}^H\dot{\mathbf{r}} =
\begin{bmatrix}
0 \\
r\dot{\theta} \\
0
\end{bmatrix}.
$$

Thus, the orbital kinematics are fully defined in the Hill frame.

The orientation of the Hill frame relative to inertial is defined using the passive 3-1-3 Euler sequence $(\Omega, i, \theta)$:

$$
[HN] = R_3(\theta)\,R_1(i)\,R_3(\Omega).
$$

Because this DCM maps inertial components into Hill components, inertial vectors are obtained using its transpose:

$$
{}^N\mathbf{r} = [HN]^T\,{}^H\mathbf{r},
\qquad
{}^N\dot{\mathbf{r}} = [HN]^T\,{}^H\dot{\mathbf{r}}.
$$

A useful geometric check of correctness is

$$
r_{N,3} = r \sin i \sin\theta,
$$

which must hold for a properly constructed DCM.

This task therefore requires us to compute the time-updated true latitude $\theta(t)$, construct the passive 3-1-3 DCM defining the Hill frame orientation, transform the canonical Hill-frame position and velocity vectors into inertial coordinates, and export the resulting inertial components in the required format.


In [9]:
def circular_orbit_rv_N(r, omega_deg, inc_deg, theta_deg, theta_dot):
    """
    Return inertial position and velocity for a circular orbit.

    Inputs
    ------
    r : float
        Orbit radius (km or m; consistent units), a vector in the orbit/hill frame
    omega_deg, inc_deg, theta_deg : float
        3-1-3 Euler angles (deg) describing the orbit frame orientation.
    theta_dot : float
        Constant true-latitude rate (rad/s) for circular orbit.

    Outputs
    -------
    r_N : (3,) ndarray
        Inertial position vector.
    v_N : (3,) ndarray
        Inertial velocity vector.
    """
    # Build [HN] = R3(theta) R1(i) R3(Omega)
    HN = ak.Euler_to_DCM(angles=[omega_deg, inc_deg, theta_deg], sequence="313", transformation_type="passive")

    NH = HN.T

    r_H = np.array([r, 0.0, 0.0])
    v_H = np.array([0.0, r * theta_dot, 0.0])

    r_N = NH @ r_H
    v_N = NH @ v_H
    return r_N, v_N

In [10]:
t_LMO = 450.0   # s
t_GMO = 1150.0  # s

# Update the true-latitude angle analytically for circular two-body motion:
#   theta(t) = theta0 + theta_dot * t
#
# Note: theta_dot is in [rad/s], so we convert the increment to degrees
# only because Euler_to_DCM() expects input angles in degrees.
theta_LMO_deg = theta0_LMO_deg + np.rad2deg(theta_dot_LMO * t_LMO)
theta_GMO_deg = theta0_GMO_deg + np.rad2deg(theta_dot_GMO * t_GMO)

# Optional: wrap to [0, 360) to keep angles tidy (does not change physics)
theta_LMO_deg = theta_LMO_deg % 360.0
theta_GMO_deg = theta_GMO_deg % 360.0

print("theta_LMO(450s)  [deg]:", theta_LMO_deg)
print("theta_GMO(1150s) [deg]:", theta_GMO_deg)

# Task 1 Parts 1 & 2
rN_LMO, vN_LMO = circular_orbit_rv_N(r_LMO_km, Omega_LMO_deg, inc_LMO_deg, theta_LMO_deg, theta_dot_LMO)
export_txt("task1_part1", rN_LMO)
export_txt("task1_part2", vN_LMO)

# Task 1 Parts 3 & 4
rN_GMO, vN_GMO = circular_orbit_rv_N(r_GMO_km, Omega_GMO_deg, inc_GMO_deg, theta_GMO_deg, theta_dot_GMO)
export_txt("task1_part3", rN_GMO)
export_txt("task1_part4", vN_GMO)

theta_LMO(450s)  [deg]: 82.81280315104198
theta_GMO(1150s) [deg]: 254.67162860463333
[Saved] outputs/task1_part1.txt -> -669.2851 3227.4983 1883.1811
[Saved] outputs/task1_part2.txt -> -3.2560 -0.7978 0.2101
[Saved] outputs/task1_part3.txt -> -5399.1504 -19697.6425 0.0000
[Saved] outputs/task1_part4.txt -> 1.3966 -0.3828 0.0000


## Task 2 - Time-Varying Hill DCM HN(t) for Circular Orbit

In [13]:
def HN_LMO_of_t(t_s: float) -> np.ndarray:
    """
    Return the passive Hill DCM [HN(t)] for the LMO orbit at time t [s].
    Uses the 3-1-3 Euler sequence (Omega, i, theta(t)).
    """
    theta_t_deg = theta0_LMO_deg + np.rad2deg(theta_dot_LMO * t_s)

    # Optional: wrap for interpretability only (not required)
    # theta_t_deg %= 360.0

    HN = ak.Euler_to_DCM(
        angles=[Omega_LMO_deg, inc_LMO_deg, theta_t_deg], 
        sequence="313",
        transformation_type="passive"
    )
    return HN

t_test = 300.0  # [s]
HN_300 = HN_LMO_of_t(t_test)

print("HN(t=300s) =\n", HN_300)
export_txt("task2_part3", HN_300, precision=6)



HN(t=300s) =
 [[-0.0464774   0.87414792  0.48343072]
 [-0.98417245 -0.12292213  0.12765086]
 [ 0.17101007 -0.46984631  0.8660254 ]]
[Saved] outputs/task2_part3.txt -> -0.046477 0.874148 0.483431 -0.984172 -0.122922 0.127651 0.171010 -0.469846 0.866025


# Module 3 - Reference Frame Orientation

# Module 4 - Attitude Evaluation and Simulator

# Module 5 - Final Mission Assessment